In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import StratifiedKFold

# Add project path
project_path = os.path.abspath("../code")
sys.path.append(project_path)
from vipm_features import ResNet50FeatureExtractor

project_path = os.path.abspath("../code")  # Adatta il percorso a dove si trova il tuo progetto
sys.path.append(project_path)
project_path = os.path.abspath("../networks")  # Adatta il percorso a dove si trova il tuo progetto
sys.path.append(project_path)
from models import *
from vipm_features import *
import vipm_costants as CONST
from vipm_pipeline import *
from dataset import *
import torch

# Configure environment variable
LOKY_MAX_CPU_COUNT = 16


In [2]:
# Carica il file CSV
def load_csv(csv_path):
    data = pd.read_csv(csv_path, header=None, names=['image_name', 'label'])
    return data['image_name'].tolist(), data['label'].tolist()

# Carica il file compresso NPZ
def load_features(npz_path):
    data = np.load(npz_path)
    features = data['features']
    labels = data['labels']
    return features, labels

# Percorsi
csv_path = '../dataset/train_small.csv'   
csv_unlabeled = '../dataset/train_unlabeled.csv'
csv_test = '../dataset/val_info.csv'

indir = '../dataset/train_set'  # Modifica in base alla posizione delle immagini
test_dir = '../dataset/val_set'
test_degraded_dir = '../dataset/val_set_degraded'

outdir = '../features'  # Modifica in base alla posizione delle feature
os.makedirs(outdir, exist_ok=True)

# Carica le features recuperate

# 20 
npz_path = os.path.join(outdir, 'features_unlabeled_retrived.npz')
features_20, labels_20 = load_features(npz_path)

# 20 cleaned
npz_path = os.path.join(outdir, 'features_unlabeled_retrived_cleaned.npz')
features_20_cleaned, labels_20_cleaned = load_features(npz_path)

print("SHAPES 20, {}, {}".format(features_20.shape, labels_20.shape))

# 40
npz_path = os.path.join(outdir, 'features_unlabeled_retrived_40.npz')
features_40, labels_40 = load_features(npz_path)

# 40 cleaned
npz_path = os.path.join(outdir, 'features_unlabeled_retrived_cleaned_40.npz')
features_40_cleaned, labels_40_cleaned = load_features(npz_path)

# 80
npz_path = os.path.join(outdir, 'features_unlabeled_retrived_80.npz')
features_80, labels_80 = load_features(npz_path)

# 80 cleaned
npz_path = os.path.join(outdir, 'features_unlabeled_retrived_cleaned_80.npz')
features_80_cleaned, labels_80_cleaned = load_features(npz_path)

# feature small 
npz_path = os.path.join(outdir, 'features_small_filtered.npz')
features_small_filtered, labels_small_filtered = load_features(npz_path)

# Carica le immagini dal CSV
image_names, labels_small = load_csv(csv_path)
labels_small = np.array(labels_small)
extractor = ResNet50FeatureExtractor()
features_small, _, _ = extractor.get_features(csv=csv_path, indir=indir, outdir=outdir, normalize=True)

# Test set 
image_names_test, labels_test = load_csv(csv_test)
labels_test = np.array(labels_test)
features_test, _, _ = extractor.get_features(csv=csv_test, indir=test_dir, outdir=outdir, normalize=True)

# Test set degraded
image_names_test_degraded, labels_test_degraded = load_csv(csv_test)
labels_test_degraded = np.array(labels_test_degraded)
features_test_degraded, _, _ = extractor.get_features(csv=csv_test, indir=test_degraded_dir, outdir=outdir, normalize=True, file_name='features_test_degraded_normalized.npz')

SHAPES 20, (5020, 2048), (1, 5020)
Caricamento delle feature da ../features\train_small_resnet50_features_normalized.npz
Caricamento delle feature da ../features\val_info_resnet50_features_normalized.npz
Caricamento delle feature da ../features\features_test_degraded_normalized.npz


In [ ]:
log_path = "results_log.csv"
log_columns = [
    "Dimension", "Small Cleaned", "Dim Cleaned", "Model", "K", "Accuracy", "Var Accuracy", "Top-5 Accuracy", "Var Top-5 Accuracy", "Top-10 Accuracy", "Var Top-10 Accuracy", "Loss", "Var Loss"
]
logs = []

# Combinazioni di dimensioni e feature
configurations = [
    {"dimension": 20, "features": features_20, "labels": labels_20, "cleaned_features": features_20_cleaned, "cleaned_labels": labels_20_cleaned},
    {"dimension": 40, "features": features_40, "labels": labels_40, "cleaned_features": features_40_cleaned, "cleaned_labels": labels_40_cleaned},
    {"dimension": 80, "features": features_80, "labels": labels_80, "cleaned_features": features_80_cleaned, "cleaned_labels": labels_80_cleaned}
]

def train_and_evaluate_knn_with_variance(X, y, k, cv_splits=5):
    knn = KNeighborsClassifier(n_neighbors=k)
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    # Cross-validation per accuracy
    accuracies = cross_val_score(knn, X, y, cv=skf, scoring='accuracy')
    mean_accuracy = np.mean(accuracies)
    var_accuracy = np.var(accuracies)

    # Cross-validation per top-5 accuracy
    top_5_accuracies = []
    top_10_accuracies = []
    
    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        knn.fit(X_train, y_train)
        top_5_predictions = np.argsort(knn.predict_proba(X_val), axis=1)[:, -5:]
        top_5_accuracy = np.mean([y in top_5 for y, top_5 in zip(y_val, top_5_predictions)])
        top_5_accuracies.append(top_5_accuracy)
        top_10_predictions = np.argsort(knn.predict_proba(X_val), axis=1)[:, -10:]
        top_10_accuracy = np.mean([y in top_10 for y, top_10 in zip(y_val, top_10_predictions)])
        top_10_accuracies.append(top_10_accuracy)

    mean_top_5_accuracy = np.mean(top_5_accuracies)
    var_top_5_accuracy = np.var(top_5_accuracies)
    
    mean_top_10_accuracy = np.mean(top_10_accuracies)
    var_top_10_accuracy = np.var(top_10_accuracies)

    return mean_accuracy, var_accuracy, mean_top_5_accuracy, var_top_5_accuracy, mean_top_10_accuracy, var_top_10_accuracy

def train_and_evaluate_neural_network_one_layer(X, y, cv_splits=5, epochs=10, batch_size=32):
    one_layer_model = OneLayerNetwork(2048, 251)
    one_layer_optimizer = torch.optim.Adam(one_layer_model.parameters(), lr=0.01)
    one_layer_scheduler = torch.optim.lr_scheduler.StepLR(one_layer_optimizer, step_size=5, gamma=0.1)
    one_layer_model_option = ModelOptions(torch.nn.CrossEntropyLoss(), one_layer_optimizer, one_layer_scheduler, input_dim = 2048)
    nn = NeuralNetwork(one_layer_model, one_layer_model_option)
    
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    accuracies = []
    top_5_accuracies = []
    top_10_accuracies = []
    losses = []

    for train_idx, val_idx in skf.split(X, y):
        # Split data into training and validation sets for this fold
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Convert to torch tensors and create datasets
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.LongTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.LongTensor(y_val)
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        
        train_loader = DataLoader(train_dataset,  batch_size=one_layer_model_option.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=one_layer_model_option.batch_size, shuffle=True)

        nn.fit(train_loader, val_loader)
        loss, top1_accuracy, top5_accuracy, top10_accuracy, y_pred_top1, y_test = nn.predict(val_loader)

        losses.append(loss)
        accuracies.append(top1_accuracy)
        top_5_accuracies.append(top5_accuracy)
        top_10_accuracies.append(top10_accuracy)


    mean_loss = np.mean(losses)
    var_loss = np.var(losses)
    
    mean_accuracy = np.mean(accuracies)
    var_accuracy = np.var(accuracies)
    
    mean_top_5_accuracy = np.mean(top_5_accuracies)
    var_top_5_accuracy = np.var(top_5_accuracies)
    
    mean_top_10_accuracy = np.mean(top_10_accuracies)
    var_top_10_accuracy = np.var(top_10_accuracies)

    return mean_accuracy, var_accuracy, mean_top_5_accuracy, var_top_5_accuracy, mean_top_10_accuracy, var_top_10_accuracy, mean_loss, var_loss

# Test small come baseline
for small_cleaned in [False, True]:
    print(f"Small Cleaned: {small_cleaned}")

    if small_cleaned:
        small_features = features_small_filtered
        small_labels = labels_small_filtered
    else:
        small_features = features_small
        small_labels = labels_small
        
    mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv, mean_loss, var_loss = train_and_evaluate_neural_network_one_layer(small_features, small_labels)
    logs.append([0, small_cleaned, False, "NN", k, mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv, mean_loss, var_loss])
    print(f"    NN: Accuracy={mean_acc_cv}, Var Accuracy={var_acc_cv}, Top-5 Accuracy={mean_top_5_acc_cv}, Var Top-5 Accuracy={var_top_5_acc_cv}, Top-10 Accuracy={mean_top_10_acc_cv}, Var Top-10 Accuracy={var_top_10_acc_cv}, Loss={mean_loss}, Var Loss={var_loss}")
        

    # Prova con diversi valori di K
    for k in [5, 10, 25, 50, 100, 150, 200]:
        print(f"  Valutazione per K={k}")

        # Cross-validation 80-20 con funzione dedicata
        mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv = train_and_evaluate_knn_with_variance(small_features, small_labels, k)
        logs.append([0, small_cleaned, False, "KNN", k, mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv, 0, 0])
        print(f"    KNN: Accuracy={mean_acc_cv}, Var Accuracy={var_acc_cv}, Top-5 Accuracy={mean_top_5_acc_cv}, Var Top-5 Accuracy={var_top_5_acc_cv}, Top-10 Accuracy={mean_top_10_acc_cv}, Var Top-10 Accuracy={var_top_10_acc_cv}")

        # # Test su test set
        # knn = KNeighborsClassifier(n_neighbors=k)
        # knn.fit(small_features, small_labels)
        # top_5_predictions_test = np.argsort(knn.predict_proba(features_test), axis=1)[:, -5:]
        # acc_test = knn.score(features_test, labels_test)
        # top_5_acc_test = np.mean([y in top_5 for y, top_5 in zip(labels_test, top_5_predictions_test)])
        # logs.append([0, small_cleaned, False, "Test Set (only on small)", k, acc_test, top_5_acc_test])
        # print(f"    Test Set (only on small): Accuracy={acc_test}, Top-5 Accuracy={top_5_acc_test}")

        # # Test su test set degraded
        # top_5_predictions_test_degraded = np.argsort(knn.predict_proba(features_test_degraded), axis=1)[:, -5:]
        # acc_test_degraded = knn.score(features_test_degraded, labels_test_degraded)
        # top_5_acc_test_degraded = np.mean([y in top_5 for y, top_5 in zip(labels_test_degraded, top_5_predictions_test_degraded)])
        # logs.append([0, small_cleaned, False, "Test Set Degraded (only on small)", k, acc_test_degraded, top_5_acc_test_degraded])
        # print(f"    Test Set Degraded (only on small): Accuracy={acc_test_degraded}, Top-5 Accuracy={top_5_acc_test_degraded}")
        
        
# Cross-validation 80-20 e test
for config in configurations:
    dim = config["dimension"]
    print(f"Iniziando configurazione per dimensione: {dim}")

    # Combinazioni di feature tra small e dimensione specifica
    for small_cleaned in [False, True]:
        for dim_cleaned in [False, True]:

            if small_cleaned:
                small_features = features_small_filtered
                small_labels = labels_small_filtered
            else:
                small_features = features_small
                small_labels = labels_small

            if dim_cleaned:
                current_features = config["cleaned_features"]
                current_labels = config["cleaned_labels"][0]
            else:
                current_features = config["features"]
                current_labels = config["labels"][0]

            print(f"  Small Cleaned: {small_cleaned}, Dim Cleaned: {dim_cleaned}")

            # Unione delle feature
            combined_features = np.concatenate((current_features, small_features), axis=0)
            combined_labels = np.concatenate((current_labels, small_labels), axis=0)

                 
            mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv, mean_loss, var_loss = train_and_evaluate_neural_network_one_layer(combined_features, combined_labels)
            logs.append([dim, small_cleaned, dim_cleaned, "NN", k, mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv, mean_loss, var_loss])
            print(f"      NN: Accuracy={mean_acc_cv}, Var Accuracy={var_acc_cv}, Top-5 Accuracy={mean_top_5_acc_cv}, Var Top-5 Accuracy={var_top_5_acc_cv}, Top-10 Accuracy={mean_top_10_acc_cv}, Var Top-10 Accuracy={var_top_10_acc_cv}, Loss={mean_loss}, Var Loss={var_loss}")

            # Prova con diversi valori di K
            for k in [5, 10, 25, 50, 100, 150]:
                print(f"    Valutazione per K={k}")

                # Cross-validation 80-20 con funzione dedicata
                mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv = train_and_evaluate_knn_with_variance(combined_features, combined_labels, k)
                logs.append([dim, small_cleaned, dim_cleaned, "KNN", k, mean_acc_cv, var_acc_cv, mean_top_5_acc_cv, var_top_5_acc_cv, mean_top_10_acc_cv, var_top_10_acc_cv, 0, 0])
                print(f"      KNN: Accuracy={mean_acc_cv}, Var Accuracy={var_acc_cv}, Top-5 Accuracy={mean_top_5_acc_cv}, Var Top-5 Accuracy={var_top_5_acc_cv}, Top-10 Accuracy={mean_top_10_acc_cv}, Var Top-10 Accuracy={var_top_10_acc_cv}")
           
                # # Validation set solo con feature della dimensione specifica
                # knn = KNeighborsClassifier(n_neighbors=k)
                # knn.fit(current_features, current_labels)
                # top_5_predictions_val = np.argsort(knn.predict_proba(small_features), axis=1)[:, -5:]
                # acc_val = knn.score(small_features, small_labels)
                # top_5_acc_val = np.mean([y in top_5 for y, top_5 in zip(small_labels, top_5_predictions_val)])
                # logs.append([dim, small_cleaned, dim_cleaned, "Validation small", k, acc_val, top_5_acc_val])
                # print(f"      Validation small: Accuracy={acc_val}, Top-5 Accuracy={top_5_acc_val}")

                # # Test su test set
                # top_5_predictions_test = np.argsort(knn.predict_proba(features_test), axis=1)[:, -5:]
                # acc_test = knn.score(features_test, labels_test)
                # top_5_acc_test = np.mean([y in top_5 for y, top_5 in zip(labels_test, top_5_predictions_test)])
                # logs.append([dim, small_cleaned, dim_cleaned, "Test Set (only on retrived)", k, acc_test, top_5_acc_test])
                # print(f"      Test Set (only on retrived): Accuracy={acc_test}, Top-5 Accuracy={top_5_acc_test}")

                # # Test su test set degraded
                # top_5_predictions_test_degraded = np.argsort(knn.predict_proba(features_test_degraded), axis=1)[:, -5:]
                # acc_test_degraded = knn.score(features_test_degraded, labels_test_degraded)
                # top_5_acc_test_degraded = np.mean([y in top_5 for y, top_5 in zip(labels_test_degraded, top_5_predictions_test_degraded)])
                # logs.append([dim, small_cleaned, dim_cleaned, "Test Set Degraded (only on retrived)", k, acc_test_degraded, top_5_acc_test_degraded])
                # print(f"      Test Set Degraded (only on retrived): Accuracy={acc_test_degraded}, Top-5 Accuracy={top_5_acc_test_degraded}")
                
                # # Training su entrambi i set
                # knn = KNeighborsClassifier(n_neighbors=k)
                # knn.fit(combined_features, combined_labels)

                # # Test su test set
                # top_5_predictions_test = np.argsort(knn.predict_proba(features_test), axis=1)[:, -5:]
                # acc_test = knn.score(features_test, labels_test)
                # top_5_acc_test = np.mean([y in top_5 for y, top_5 in zip(labels_test, top_5_predictions_test)])
                # logs.append([dim, small_cleaned, dim_cleaned, "Test Set (on small and retrived)", k, acc_test, top_5_acc_test])
                # print(f"      Test Set (on small and retrived): Accuracy={acc_test}, Top-5 Accuracy={top_5_acc_test}")

                # # Test su test set degraded
                # top_5_predictions_test_degraded = np.argsort(knn.predict_proba(features_test_degraded), axis=1)[:, -5:]
                # acc_test_degraded = knn.score(features_test_degraded, labels_test_degraded)
                # top_5_acc_test_degraded = np.mean([y in top_5 for y, top_5 in zip(labels_test_degraded, top_5_predictions_test_degraded)])
                # logs.append([dim, small_cleaned, dim_cleaned, "Test Set Degraded (on small and retrived)", k, acc_test_degraded, top_5_acc_test_degraded])
                # print(f"      Test Set Degraded (on small and retrived): Accuracy={acc_test_degraded}, Top-5 Accuracy={top_5_acc_test_degraded}")

# Salva i log in un CSV
log_df = pd.DataFrame(logs, columns=log_columns)
log_df.to_csv(log_path, index=False)
print(f"Log salvato in {log_path}")

Small Cleaned: False
  Valutazione per K=5


c:\Users\sguid\miniconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\sguid\miniconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


    KNN: Accuracy=0.14003984063745017, Var Accuracy=0.0002200758718115585, Top-5 Accuracy=0.3264940239043824, Var Top-5 Accuracy=0.0001061094268344945, Top-10 Accuracy=0.33944223107569715, Var Top-10 Accuracy=9.8966683068523e-05
Epoch 1/100:
  Train Loss: 5.6594, Train Accuracy: 0.35%
  Val Loss: 5.5262, Val Accuracy: 0.40%
Epoch 2/100:
  Train Loss: 5.5834, Train Accuracy: 0.45%
  Val Loss: 5.5259, Val Accuracy: 0.60%
Epoch 3/100:
  Train Loss: 5.4826, Train Accuracy: 1.02%
  Val Loss: 5.5162, Val Accuracy: 1.39%
Epoch 4/100:
  Train Loss: 5.3444, Train Accuracy: 1.59%
  Val Loss: 5.4605, Val Accuracy: 1.49%
Epoch 5/100:
  Train Loss: 5.2067, Train Accuracy: 1.62%
  Val Loss: 5.3442, Val Accuracy: 1.39%
Epoch 6/100:
  Train Loss: 5.0918, Train Accuracy: 2.14%
  Val Loss: 5.2234, Val Accuracy: 1.59%
Epoch 7/100:
  Train Loss: 5.0823, Train Accuracy: 2.17%
  Val Loss: 5.3136, Val Accuracy: 1.00%
Epoch 8/100:
  Train Loss: 5.0556, Train Accuracy: 2.61%
  Val Loss: 5.3136, Val Accuracy: 1